In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  


In [2]:
chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome('chromedriver', options=chrome_options)

### Extract date



In [3]:
url = 'https://www.moh.gov.sg/covid-19'

driver.get(url)
content = driver.page_source


In [4]:
soup =BeautifulSoup(content, 'lxml')

In [5]:
data = soup.findAll(text=re.compile("Case Summary in Singapore \(as of .+"))


In [6]:
data

['Case Summary in Singapore (as of 29 Apr 2020, 1200h)']

In [7]:
m = re.search("(\d+ \w+ 202\d, \d+\d+h)", data[0])
date_time_str = m.group(1)
data_date_time = datetime.strptime(date_time_str, '%d %b %Y, %H%Mh').strftime('%Y-%m-%d')
data_date_time


'2020-04-29'

### Extract Dataframes

In [8]:
dfs = pd.read_html(content)

In [9]:
dfs

[    0   1
 0 NaN NaN
 1 NaN NaN,
     0   1
 0 NaN NaN
 1 NaN NaN,
                0       1
 0  DORSCON Level  Orange,
           0         1         2
 0  IMPORTED  IMPORTED  IMPORTED
 1  571 (+0)  571 (+0)  571 (+0),
                0
 0  Active Cases#
 1           1714,
              0
 0  Discharged^
 1         1188,
                           0
 0  In Isolation Facilities*
 1                     12725,
                        0
 0  Hospitalised (Stable)
 1                   1692,
                          0
 0  Hospitalised (Critical)
 1                       22,
         0
 0  Deaths
 1      14,
         0
 0  143919,
        0
 0  99929,
          0
 0  ~25,200,
          0
 0  ~17,500,
               0                                                  1
 0          Date                                              Title
 1   29 Apr 2020  62 More Cases Discharged; 690 New Cases of COV...
 2   29 Apr 2020  ERP Charging and COE Bidding Continue to be Su...
 3   29 Apr 2020  Coord

In [10]:
df_idx = 0
for idx in range(len(dfs)):
    if 'Active' in str(dfs[idx][0][0]):
        print("found starting df")
        df_idx = idx
        break
print(df_idx)
assert df_idx < len(dfs)

found starting df
4


In [11]:
dfs[df_idx]

,0
0,Active Cases#
1,1714


In [12]:
active=df_idx
assert dfs[active][0][0].strip().startswith('Active Cases')
dfs[active]


,0
0,Active Cases#
1,1714


In [13]:
discharge = df_idx+1
assert dfs[discharge][0][0].strip().startswith('Discharged')
dfs[discharge]

,0
0,Discharged^
1,1188


In [14]:
hospitalised_stable = df_idx+3
assert dfs[hospitalised_stable][0][0].strip().startswith('Hospitalised (Stable)')

dfs[hospitalised_stable]

,0
0,Hospitalised (Stable)
1,1692


In [15]:
hospitalised_critical = df_idx+4
assert dfs[hospitalised_critical][0][0].strip().startswith('Hospitalised (Critical)')

dfs[hospitalised_critical]

,0
0,Hospitalised (Critical)
1,22


In [16]:
deaths = df_idx+5
assert dfs[deaths][0][0].strip().startswith('Deaths')

dfs[deaths]

,0
0,Deaths
1,14


In [17]:
# # Look for df with confirmed cases, tested negative, ...

# for df in dfs:
#     combined_fields = df[3].to_string().lower()
#     if "active cases" in combined_fields:
#         sing_cases_df_temp = df
        
# sing_cases_df_temp

In [18]:
def remove_non_numeric(s):
    return re.sub('\D','', s)


In [19]:
# very hacky to workaround with the change from MoH site
sing_cases_df = pd.DataFrame({'active_cases': [int(remove_non_numeric(dfs[active][0][1]))],
                             'hospitalised_stable': [int(remove_non_numeric(dfs[hospitalised_stable][0][1]))],
                              'hospitalised_critical': [int(remove_non_numeric(dfs[hospitalised_critical][0][1]))],
                              'death': [int(remove_non_numeric(dfs[deaths][0][1]))],
                              'discharge': [int(dfs[discharge][0][1])],
                              'datetime': data_date_time
                             },
                            )
sing_cases_df

,active_cases,hospitalised_stable,hospitalised_critical,death,discharge,datetime
0,1714,1692,22,14,1188,2020-04-29


In [20]:
df = sing_cases_df
df

,active_cases,hospitalised_stable,hospitalised_critical,death,discharge,datetime
0,1714,1692,22,14,1188,2020-04-29


### Save to file

In [21]:
from pathlib import Path
csv_file = '../data/singapore-cases.csv'

if Path(csv_file).exists():
    # read out the old data
    old_df = pd.read_csv(csv_file)
    df = df.set_index('datetime').append(old_df.set_index('datetime'))



/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [22]:
df = df.reset_index()
df.drop_duplicates(subset=['datetime'], inplace=True)

In [23]:
df.sort_values(by=['datetime'], inplace=True)
df

,datetime,active_cases,confirmed,death,discharge,discharged,hospitalised,hospitalised_critical,hospitalised_stable,negative,pending
1,2020-02-02,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,240.0,43.0
2,2020-02-03,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,262.0,32.0
3,2020-02-04,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,289.0,20.0
4,2020-02-05,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN,295.0,62.0
5,2020-02-06,NaN,33.0,NaN,NaN,NaN,NaN,NaN,NaN,310.0,147.0
...,...,...,...,...,...,...,...,...,...,...,...
84,2020-04-25,1214.0,NaN,12.0,1002.0,NaN,NaN,24.0,1190.0,NaN,NaN
85,2020-04-26,1311.0,NaN,12.0,1060.0,NaN,NaN,22.0,1289.0,NaN,NaN
86,2020-04-27,1451.0,NaN,142.0,1095.0,NaN,NaN,20.0,1431.0,NaN,NaN
87,2020-04-28,1689.0,NaN,14.0,1128.0,NaN,NaN,21.0,1668.0,NaN,NaN


In [24]:
df.to_csv(csv_file, index=False)
